In [ ]:
import findspark
findspark.init('/usr/local/spark')
import pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, RegexTokenizer, StopWordsRemover, CountVectorizerModel, CountVectorizer

In [ ]:
from pyspark.ml.linalg import SparseVector, Vector

In [ ]:
spark = SparkSession.builder.appName("PySpark Text Analysis Example").getOrCreate()

In [ ]:
sc = spark.sparkContext

In [ ]:
# read all the files
wikif = sc.wholeTextFiles("file:///home/hduser/Downloads/sharedfolder/mlexamples/TFIDF-CaseStudy-4-PySpark/wikidocs/*.txt")
# wikif = sc.wholeTextFiles("mllib/wikidocs/*.txt")

In [ ]:
wikif.first()

In [ ]:
# Remove path from the file name
# wikif.first()[0].split('/')[-1]
# (wikif.first()[0].split('/')[-1],wikif.first()[1])

wiki=wikif.map(lambda rec:((rec[0].split('/')[-1]),rec[1]))

In [ ]:
wiki.first()

In [ ]:
# convert to dataframe
filedata=spark.createDataFrame(wiki).toDF('label','doc')
filedata.printSchema()

In [ ]:
filedata.show(2)

In [ ]:
# convert document to list of words
tokenizer = RegexTokenizer(inputCol="doc", outputCol="allwords", pattern="\\W")
allWordsData = tokenizer.transform(filedata)
allWordsData.printSchema()

In [ ]:
allWordsData.show(2)

In [ ]:
# remove the stop words
remover = StopWordsRemover(inputCol="allwords", outputCol="words")
wordsData=remover.transform(allWordsData)
wordsData.printSchema()

In [ ]:
wordsData.show(2)

In [ ]:
wordsData.select('allwords').show(1,False)

In [ ]:
wordsData.select('words').show(1,False)

In [ ]:
# Get the required columns 
nwordsData=wordsData.select('label','words')
nwordsData.printSchema()

In [ ]:
nwordsData.show(2)

In [ ]:
# Build a term frequency matrix. Also check the vocabulary
# val cvModel: CountVectorizerModel = new CountVectorizer().setInputCol("words").setOutputCol("rawFeatures").fit(nwordsData)
# cvModel.vocabulary.length
# val cvm = cvModel.transform(nwordsData)
cvModel=CountVectorizer(inputCol='words',outputCol='rawFeatures').fit(nwordsData)
len(cvModel.vocabulary)

In [ ]:
type(cvModel.vocabulary)

In [ ]:
cvModel.vocabulary

In [ ]:
cvm = cvModel.transform(nwordsData)
type(cvm)

In [ ]:
cvm.printSchema()

In [ ]:
cvm.show(2)

In [ ]:
# Build the Inverse document frequency
idf=IDF(inputCol="rawFeatures", outputCol="features").fit(cvm)
tfIDFData = idf.transform(cvm)

In [ ]:
# Apply the function to extract the top words.
# Top 10 is specified below but you can change that.
tfIDFData.printSchema()

In [ ]:
tfIDFData.show(2)

In [ ]:
tfIDFData.count()

In [ ]:
tfIDFRDD=tfIDFData.select("label","features").rdd

In [ ]:
tfIDFRDD.count()

In [ ]:
type(tfIDFRDD.first())

In [ ]:
tfIDFRDD.first()

In [ ]:
tfIDFRDD.first().features.indices

In [ ]:
len(tfIDFRDD.first().features.indices)

In [ ]:
tfIDFRDD.first().features.values

In [ ]:
len(tfIDFRDD.first().features.values)

In [ ]:
type(tfIDFRDD.first().features.values[0])

In [ ]:
type(tfIDFRDD.first().features.indices[0])

In [ ]:
tfIDFRDD.first().label

In [ ]:
def findf2(vocab,words,idfs):
    dict1={}
    for i in range(len(words)):
        if idfs[i] in dict1:
            dict1[idfs[i]].append(words[i])
        else:
            dict1[idfs[i]]=[words[i]]
    list1=list(dict1.keys())
    list1.sort(reverse=True)
    kwords=[]
    for j in list1[:10]:
        for k in range(len(dict1[j])):
            kwords.append(vocab[dict1[j][k]])
    return kwords[:10]
 

In [ ]:
for rec in tfIDFRDD.take(tfIDFRDD.count()):
    kwords10=(rec.label, findf2(cvModel.vocabulary,rec.features.indices, rec.features.values))
    print(kwords10)
    